In [ ]:
import json
import pandas as pd
import os
import numpy as np


import folium
from folium import plugins
from pprint import pprint
from folium.plugins import HeatMap

import plotly.graph_objects as go
import cufflinks as cf
cf.go_offline(connected=True)

import requests
from urllib.parse import urlparse

In [ ]:
# 데이터 파일 병합
# PATH = 'G:/Repository/PublicData/Jingu-Project/Parking/data/parkinglot'
# os.chdir(PATH)
os.getcwd() # 현재 위치

df_list = list(os.listdir(os.getcwd()))

df_merge = pd.DataFrame()
for d in df_list:
    df = pd.read_csv(d, encoding='euc-kr')
    df_merge = pd.concat([df_merge, df], ignore_index=True)

df_merge.info()

In [ ]:
# 병합 데이터 저장
df_merge.to_csv('../부산진구 주자창 현황(2021.11.).csv', encoding='euc-kr', index=False)

In [ ]:
# 병합 데이터 가져오기
df_merge = pd.read_csv('./data/부산진구 주자창 현황(2021.11.).csv', encoding='euc-kr')
df_merge.head()

In [ ]:
# 데이터 컬럼
df_merge.columns

In [ ]:
# 대충 필요해보이는 컬럼
df_merge[['주차장구분', '주차장유형', '주차장명', '소재지지번주소', '주차구획수']]

In [ ]:
# 주차구획 to int
df_merge['주차구획수'].notnull()

In [ ]:
# 각 열의 유니크 값
# df_merge['주차장구분'].unique()
df_merge['주차장유형'].unique()

In [ ]:
# 주차장구분
df1 = df_merge[['주차장구분', '주차구획수']]
df1 = df1.groupby('주차장구분').sum()
df1.rename(columns={'주차구획수': '빈도'}, inplace=True)
df1

layout_kind = {
    'title': {
        'text': '<b>주차장구분에 따른 주차구획수 파이차트 분석</b>',
        'font': {
            'size': 20
        },
        'x': 0.5,
    },
    'showlegend':True,
    'legend': {
        'title': {
            'text': '<b>주차장구분</b>',
            'font': {
                'size': 15
            },
        },
        'borderwidth':25, 
        'bordercolor': '#fff',
        'bgcolor': '#F0F8FF',
        'orientation': 'h',
    },
    'autosize': True,
    # 'width': 600,
    # 'height': 600,
}

kind = go.Figure()

kind.add_trace(
    go.Pie(
        labels=df1.index,
        values=df1['빈도'],
    )
)

kind.update_traces(
    go.Pie(
        textposition='inside',
        textinfo='label+value+percent',
        textfont = {
            'size': 15,
        }
    )
)

kind.update_layout(layout_kind)
kind.show()

In [ ]:
# 주차장 유형
df2 = df_merge[['주차장유형', '주차장구분']]
df2 = df2.groupby('주차장유형').count()
df2.rename(columns={'주차장구분': '빈도'}, inplace=True)

df3 = df_merge[['주차장유형', '주차구획수']]
df3 = df3.groupby('주차장유형').sum()
df3.rename(columns={'주차구획수': '빈도'}, inplace=True)
df3

layout = {
    'title': {
        'text': '<b>주차장유형에 따른 주차구획수 파이차트 분석</b>',
        'font': {
            'size': 20
        },
        'x': 0.5,
    },
    'showlegend':True,
    'legend': {
        'title': {
            'text': '<b>주차장유형</b>',
            'font': {
                'size': 15
            },
        },
        'borderwidth':25, 
        'bordercolor': '#fff',
        'bgcolor': '#F0F8FF',
        'orientation': 'h',
        'x':0.15
    },
    'autosize': True,
    # 'width': 600,
    # 'height': 600,
}

kind = go.Figure()

kind.add_trace(
    go.Pie(
        labels=df3.index,
        values=df3['빈도'],
    )
)

kind.update_traces(
    go.Pie(
        textposition='inside',
        textinfo='label+value+percent',
        textfont = {
            'size': 15,
        }
    )
)

kind.update_layout(layout_kind)
kind.show()

In [ ]:
# 주차장 구획수
df4 = df_merge[['주차장명', '주차구획수']]
df4 = df4.groupby('주차장명', as_index=False).sum()
df4.rename(columns={'주차구획수': '빈도'}, inplace=True)
df4.sort_values('빈도', ascending=False, inplace=True)
df4.set_index('주차장명', inplace=True)
df4_ = df4[1:6]
df4_

In [ ]:
layout_mobile = {
    'title': {
        'text': '<b>[주차구획수]</b> Top5',
        'x': 0.5,
        'font': {
            'size': 20
        }
    },
    'showlegend': False,
    'xaxis': {
        'title': '접수기관',


    },
    'yaxis': {
        'title': '접수 민원 수',
        'showticklabels': True,
        'dtick': 10000
    }
}

mobile = go.Figure()

mobile.add_trace(
    go.Bar(
        x=df4_.index,
        y=df4_['빈도'], name='접수기관',
        text=df4_['빈도'], textposition='auto',
    )
)

mobile.update_layout(layout_mobile)
mobile.show()


In [ ]:
# 주차장 현황 지도
df5 = df_merge[['주차장명', '소재지도로명주소', '소재지지번주소', '주차구획수']]
df5

In [ ]:
# 초기화
df5['주소'] = np.nan

In [ ]:
# 주소 넣기
for i in range(len(df5)):
    if df5['소재지도로명주소'][i]:
        df5['주소'][i] = df5['소재지도로명주소'][i]
    elif df5['소재지지번주소'][i]:
        df5['주소'][i] =  df5['소재지지번주소'][i]
    else:
        df5['주소'][i] =  ""

In [ ]:
df5 = df5[['주차장명', '주소', '주차구획수']]
df5.head()

In [ ]:
df5.dropna(inplace=True)
df5.drop_duplicates(inplace=True)
df5.reset_index(drop=True, inplace=True)
df5.info()

In [ ]:
df5.head()

In [ ]:
# 주소 검색 함수 정의
def address_to_latlon(address):
    url = "https://dapi.kakao.com/v2/local/search/address.json?&query=" + address
    result = requests.get(urlparse(url).geturl(),
                          headers={"Authorization":"KakaoAK APIKey"})
    json_obj = result.json()
    keys = [key for key in json_obj]
    if keys[0] == 'documents':
        for document in json_obj['documents']:
            val = [document['address_name'], document['y'], document['x']]
            return val
        return ["", "", ""]
    else:
        return ["", "", ""]

In [ ]:
# 위도/경도 붙일 빈 열 만들기
df5_ = df5
df5_['위도'] = np.nan
df5_['경도'] = np.nan

In [ ]:
# 위도, 경도 구하기
for i in range(len(df5_)):
    min_address = df5_['주소'][i]
    tmp = address_to_latlon(min_address)
    df5_['위도'][i] = tmp[1]
    df5_['경도'][i] = tmp[2]

In [ ]:
df5_.to_csv('./data/result/주차장.csv', encoding='euc-kr', index=False)

In [ ]:
df5_ = pd.read_csv('./data/result/주차장.csv', encoding='euc-kr')
# df5_ = df5_[['주차장명', '주소', '주차구획수', '위도', '경도']]
df5_.head()

In [ ]:
df5_.info()
df5_.head()

In [ ]:
df5_['위도'] = pd.to_numeric(df5_['위도'])
df5_['경도'] = pd.to_numeric(df5_['경도'])
df5_['주차구획수'] = df5_['주차구획수'].astype(int)

In [ ]:
df5_[df5_['위도'].isnull()]

In [ ]:
df5_.dropna(inplace=True)
df5_[df5_['위도'].isnull()]

In [ ]:
min_address = "부산광역시 부산진구 가야공원로 74"
tmp = address_to_latlon(min_address)
tmp

In [ ]:
df5_.head()

In [ ]:
df5_.columns

In [ ]:
df5_.reset_index(drop=True, inplace=True)

In [ ]:
df5_

In [ ]:
# shp 파일
state_geo = './data/지도/TL_SCCO_SIG_WGS84.json'

# json 파일 로드
with open(state_geo, encoding='utf-8') as file:
    json_file = json.load(file)

In [ ]:
# 민원 발생 주소 버블 차트
lat_mean = df5_['위도'].mean()
lon_mean = df5_['경도'].mean()

top_m = folium.Map(
    location=[lat_mean, lon_mean],
    zoom_start=13)

folium.Choropleth(
    json_file,
    fill_opacity=0.1,
    fill_color='#3186cc',
    line_opacity=1,
    line_weight=0.3,
    line_color='#3186cc'
    ).add_to(top_m)


for i in range(len(df5_)):
    latitude = df5_['위도'][i]
    longitude = df5_['경도'][i]
    radius = df5_['주차구획수'][i]/100
    text = df5_['주차장명'][i]
    location = (latitude, longitude)
    folium.CircleMarker(
        location,
        radius=radius,
        color='#3186cc',
        fill_color='#3186cc',
        opacity=0.7,
        popup=text).add_to(top_m)
    


top_m


In [ ]:
# 히트맵

lat_mean = df5_['위도'].mean()
lon_mean = df5_['경도'].mean()


heat_m = folium.Map(
    location=[lat_mean, lon_mean],
    tiles="cartodbpositron",
    zoom_start=11)

folium.Choropleth(
    json_file,
    fill_opacity=0.1,
    fill_color='#20B2AA',
    line_opacity=0.3,
    line_weight=0.9,
    line_color='#20B2AA').add_to(heat_m)

heat_df = df5_[['위도', '경도']]

heat_data = [[row['위도'],row['경도']] for index, row in heat_df.iterrows()]

HeatMap(heat_data).add_to(heat_m)

heat_m
